In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import include.Model_and_Methods as mm
import os
from os.path import join
from torch.utils.data import DataLoader
import pandas as pd
import seaborn as sn

In [2]:
device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "cpu"
)
torch.set_default_device(device)

In [3]:
def load_result_dicts(directory, strategy_name, num_seeds, num_total_expansions, eps, lr, grad_clip,
                      initial_network_id):
    result_dict_list = []
    start_seed = 1
    for seed in range(start_seed, num_seeds + start_seed):
        if num_total_expansions > 0:
            result_dict_list.append(torch.load(
                directory + "/" + strategy_name + f"_result_seed_{seed}_expansions_{num_total_expansions}_eps_{eps}_lr_{lr}_grad_clip_{grad_clip}_initial_network_{initial_network_id}.pt"))
        else:
            added_name = "_with_loss_convergence_termination" # remove this from the path name below to obtain the baseline model trained for a fixed number of epochs(=50) 
            result_dict_list.append(torch.load(
                directory + "/" + strategy_name + f"_result_seed_{seed}_lr_{lr}_grad_clip_{grad_clip}.pt"))  #{added_name}
    return result_dict_list

In [4]:
def get_result_list_end_epoch(list, training_data_size=60000, log_interval=100,
                              batch_size=100):  # does not hold for "index_before_expand"
    stepsize = int(training_data_size / (log_interval * batch_size))
    return list[stepsize - 1::stepsize]

In [5]:
# def convert_list_to_numpy(list, elements_are_tensor):
#     if elements_are_tensor:
#         list = [list[i].item() for i in range(len(list))]
#     return np.array(list)

In [6]:
def tolerant_mean(arrs):
    lens = [len(i) for i in arrs]
    arr = np.ma.empty((np.max(lens), len(arrs)))
    arr.mask = True  # represents missing value
    for idx, l in enumerate(arrs):
        arr[:len(l), idx] = l
    return arr.mean(axis=-1), arr.std(axis=-1)

In [7]:
def tensor_mean(
        tensor_list):  # can not handle missing values, suited for multi-dimensional arrays (matrices, tensors, etc)
    mean_tensor = np.zeros_like(tensor_list[0])
    for i in range(len(tensor_list)):
        mean_tensor += tensor_list[i]
    mean_tensor = mean_tensor / len(tensor_list)
    return mean_tensor

In [8]:
def get_params_layers_expansions(mask_dict_list):  # can not handle missing values
    num_expansions = len(mask_dict_list)
    num_layers = len(mask_dict_list[0]["weights"])

    accumulated_params_added_to_layers = [[(torch.count_nonzero(
        mask_dict_list[expansion]["weights"][layer]).item() + torch.count_nonzero(
        mask_dict_list[expansion]["biases"][layer]).item()) for layer in range(num_layers)] for expansion in
                                          range(num_expansions)]  # accumulated
    accumulated_params_added_to_layers.append(
        [(torch.numel(mask_dict_list[0]["weights"][layer]) + torch.numel(mask_dict_list[0]["biases"][layer])) for layer
         in range(num_layers)])

    params_added_to_layers = [[(torch.count_nonzero(
        mask_dict_list[expansion]["weights"][layer]).item() + torch.count_nonzero(
        mask_dict_list[expansion]["biases"][layer]).item()) for layer in range(num_layers)] for expansion in
                              range(num_expansions)]  # non accumulated
    params_added_to_layers.append(
        [(torch.numel(mask_dict_list[0]["weights"][layer]) + torch.numel(mask_dict_list[0]["biases"][layer])) for layer
         in range(num_layers)])

    for e in range(num_expansions + 1):
        for l in range(num_layers):
            if e != 0:
                params_added_to_layers[e][l] -= accumulated_params_added_to_layers[e - 1][l]

    return params_added_to_layers

In [9]:
def plot_with_std_expansions_baseline(ax, x, y, y_std, label, epoch_before_expand,
                                      mean_baseline_acc, num_params_mean_arr, num_total_params, flag_only_baseline_peak,
                                      acc_means_random_baseline):  # for loss and accuracy
    ax.plot(x, y, color="blue", label=label)
    ax.plot(np.arange(1, acc_means_random_baseline.size + 1), acc_means_random_baseline, color="orange",
            label="random strategy")
    ax.fill_between(x, y + y_std, y - y_std, facecolor='blue', alpha=0.5)
    # ax.vlines(x=epoch_before_expand, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], colors="green",
    #           linestyles="dashed", label="Expansion", alpha=0.3)
    # print(epoch_before_expand)
    relative_num_params_at_expansion_epoch = [100 * num_params_mean_arr[int(i)] / num_total_params for i in
                                              epoch_before_expand]  # epoch after expansion  
    ax.bar(epoch_before_expand, relative_num_params_at_expansion_epoch, color="green", alpha=0.5)
    if flag_only_baseline_peak:
        ax.hlines(y=np.max(mean_baseline_acc), xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], colors="red",
                  linestyles="dashed",
                  label="Baseline Peak", alpha=0.3)
    else:
        ax.plot(np.arange(1, len(mean_baseline_acc) + 1), mean_baseline_acc, color="red", label="baseline")

In [41]:
num_expansions = 5 #TODO
eps = 0.01
grad_clip = 100
lr = 0.001
initial_network_id = 2
num_seeds = 5
fashion_mnist_name = "Fashion_MNIST"
mnist_name = "MNIST"
dataset_name = fashion_mnist_name
strategies_result_dict = dict()
strategies_result_dict["random_edges"] = load_result_dicts("../results/strategy1_random/edges" + "/" + dataset_name,
                                                           "random_edges", num_seeds, num_expansions, eps, lr,
                                                           grad_clip, initial_network_id)
strategies_result_dict["random_neurons"] = load_result_dicts("../results/strategy1_random/neurons" + "/" + dataset_name,
                                                             "random_neurons", num_seeds, num_expansions,
                                                             eps, lr, grad_clip, initial_network_id)
strategies_result_dict["constant_edges"] = load_result_dicts("../results/strategy2_constant/edges" + "/" + dataset_name,
                                                             "constant_edges", num_seeds, num_expansions,
                                                             eps, lr, grad_clip, initial_network_id)
strategies_result_dict["constant_neurons"] = load_result_dicts(
    "../results/strategy2_constant/neurons" + "/" + dataset_name, "constant_neurons", num_seeds,
    num_expansions, eps, lr, grad_clip, initial_network_id)
strategies_result_dict["regularized_edges"] = load_result_dicts(
    "../results/strategy3_regularized/edges" + "/" + dataset_name, "regularized_edges",
    num_seeds, num_expansions,
    eps, lr, grad_clip, initial_network_id)
strategies_result_dict["regularized_neurons"] = load_result_dicts(
    "../results/strategy3_regularized/neurons" + "/" + dataset_name,
    "regularized_neurons", num_seeds,
    num_expansions, eps, lr, grad_clip, initial_network_id)
strategies_result_dict["initial_edges"] = load_result_dicts("../results/strategy4_initial/edges" + "/" + dataset_name,
                                                            "initial_edges", num_seeds, num_expansions, eps, lr,
                                                            grad_clip, initial_network_id)
strategies_result_dict["initial_neurons"] = load_result_dicts(
    "../results/strategy4_initial/neurons" + "/" + dataset_name, "initial_neurons", num_seeds, num_expansions, eps, lr,
    grad_clip, initial_network_id)

strategies_result_dict["warmstarted_edges"] = load_result_dicts(
    "../results/strategy4_warmstarted/edges" + "/" + dataset_name, "warmstarted_edges", num_seeds, num_expansions, eps,
    lr, grad_clip, initial_network_id)
strategies_result_dict["warmstarted_neurons"] = load_result_dicts(
    "../results/strategy4_warmstarted/neurons" + "/" + dataset_name, "warmstarted_neurons", num_seeds, num_expansions,
    eps, lr, grad_clip, initial_network_id)

strategies_result_dict["gradient_based_edges"] = load_result_dicts(
    "../results/strategy5_gradient_based/edges" + "/" + dataset_name, "gradient_based_edges", num_seeds, num_expansions,
    eps, lr, grad_clip, initial_network_id)
strategies_result_dict["gradient_based_neurons"] = load_result_dicts(
    "../results/strategy5_gradient_based/neurons" + "/" + dataset_name, "gradient_based_neurons", num_seeds,
    num_expansions, eps, lr, grad_clip, initial_network_id)

strategies_result_dict["layer_stat_edges"] = load_result_dicts(
    "../results/strategy6_layer_stat/edges" + "/" + dataset_name, "layer_stat_edges", num_seeds, num_expansions, eps,
    lr, grad_clip, initial_network_id)
strategies_result_dict["layer_stat_neurons"] = load_result_dicts(
    "../results/strategy6_layer_stat/neurons" + "/" + dataset_name, "layer_stat_neurons", num_seeds, num_expansions,
    eps, lr, grad_clip, initial_network_id)

# strategies_result_dict["splitting_neurons_eigenvectors_init"] = load_result_dicts("../results/strategy5_splitting/neurons" + "/" + dataset_name, "splitting_neurons_eigenvectors_init", num_seeds, num_expansions, eps)

# result_dict_list_layer_statistics_edges = load_result_dicts("../results/strategy6_layer_statistics/edges", "layer_statistics_edges", num_seeds, num_expansions, eps)
# result_dict_list_layer_statistics_neurons = load_result_dicts("../results/strategy6_layer_statistics/neurons", "layer_statistics_neurons", num_seeds, num_expansions, eps)

strategies_result_dict["baseline"] = load_result_dicts("../results/baseline" + "/" + dataset_name, "baseline",
                                                       num_seeds, 0, 0, lr, grad_clip, 0)

In [42]:
num_total_params = strategies_result_dict["random_edges"][0]["num_params"][-1]

# evaluating only one entry per epoch

strategies_test_accuracy = dict()
strategies_epoch_before_expand = dict()
strategies_num_params = dict()
strategies_num_params_layers_expansions = dict()

for strategy in strategies_result_dict.keys():
    strategies_test_accuracy[strategy] = []

    if strategy != "baseline":
        strategies_epoch_before_expand[strategy] = []
        strategies_num_params[strategy] = []
        strategies_num_params_layers_expansions[strategy] = []

for seed in range(num_seeds):
    for strategy in strategies_result_dict.keys():
        strategies_test_accuracy[strategy].append(
            np.array(get_result_list_end_epoch(strategies_result_dict[strategy][seed]["test_accuracy"])))
        if strategy != "baseline":
            strategies_epoch_before_expand[strategy].append(
                (np.array(strategies_result_dict[strategy][seed]["index_before_expand"]) + 1) / 6)
            strategies_num_params[strategy].append(
                np.array(get_result_list_end_epoch(strategies_result_dict[strategy][seed]["num_params"])))
            strategies_num_params_layers_expansions[strategy].append(
                np.array(get_params_layers_expansions(strategies_result_dict[strategy][seed]["masks"])))

acc_means = dict()
acc_stds = dict()
num_params_means = dict()
expansion_indices_means = dict()
expansion_indices_stds = dict()
num_params_layers_expansions_means = dict()

for strategy in strategies_result_dict.keys():
    acc, std = tolerant_mean(strategies_test_accuracy[strategy])
    acc_means[strategy] = acc
    acc_stds[strategy] = std
    if strategy != "baseline":
        num_params_means[strategy] = tolerant_mean(strategies_num_params[strategy])[0]
        expansion_indices_means[strategy], expansion_indices_stds[strategy] = tolerant_mean(strategies_epoch_before_expand[strategy])
        num_params_layers_expansions_means[strategy] = tensor_mean(strategies_num_params_layers_expansions[strategy])

In [43]:
print("mean training duration: ", np.mean([len(run) for run in strategies_test_accuracy["baseline"]]))
print("mean accuracy: ", np.mean(acc_means["baseline"]))
print("final accuracy: ", acc_means["baseline"][-1])

mean training duration:  50.0
mean accuracy:  86.08439999999999
final accuracy:  86.244


In [34]:
max_epoch = np.max([len(i) for i in acc_means.values()])
print("max epoch: ", max_epoch)

fig = plt.figure(figsize=(4 * 6.4, 4 * 4.8), layout="constrained")  # 4 * 6.4, 3 * 4.8

for i, strategy in enumerate(acc_means, start=1):  # iterates over keys which are the same for each result dict
    if strategy != "baseline":
        ax_i = fig.add_subplot(4, 4, i)  # 3, 4, i
        ax_i.set_title("Accuracy " + strategy)
        ax_i.set_xlabel("Epoch")
        ax_i.set_ylabel("Accuracy (%)")
        ax_i.set_xlim((0, max_epoch))
        ax_i.set_ylim((0, 100))
        if "edges" in strategy:
            plot_with_std_expansions_baseline(ax_i, np.arange(1, len(acc_means[strategy]) + 1), acc_means[strategy],
                                              acc_stds[strategy], "Test-Accuracy",
                                              expansion_indices_means[strategy], acc_means["baseline"],
                                              num_params_means[strategy], num_total_params, False,
                                              acc_means["random_edges"])
        else:
            plot_with_std_expansions_baseline(ax_i, np.arange(1, len(acc_means[strategy]) + 1), acc_means[strategy],
                                              acc_stds[strategy], "Test-Accuracy",
                                              expansion_indices_means[strategy], acc_means["baseline"],
                                              num_params_means[strategy], num_total_params, False,
                                              acc_means["random_neurons"])
    plt.legend()
plt.show()
fig.savefig(
    f'../plots/{dataset_name}/Accuracies_Many_Plots_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.pdf',
    bbox_inches="tight")  # save the figure to file
plt.close(fig)

max epoch:  50


In [35]:
colors = ["blue", "green", "cyan", "purple", "orange", "brown", "magenta", ]
# alpha = 0.5
fig = plt.figure(figsize=(6.4, 6.4), layout="constrained")
ax = fig.add_subplot(111)
# ax.set_title("Test Accuracies der Strategien in Kantenperspektive")
ax.set_xlabel("Epoch")
ax.set_ylabel("Accuracy (%)")
ax.set_xlim((0, max_epoch))
ax.set_ylim((0, 100))
curr_strat_index = 0
for i, strategy in enumerate(acc_means):  # iterates over keys which are the same for each result dict
    if "edges" in strategy:
        ax.plot(np.arange(1, len(acc_means[strategy]) + 1), acc_means[strategy], label=strategy,
                color=colors[curr_strat_index], alpha=0.5)  #, alpha= alpha
        curr_strat_index += 1
        # if alpha == 0.5:
        #     alpha -= 0.25
        # else:
        #     alpha += 0.25
ax.plot(np.arange(1, len(acc_means["baseline"]) + 1), acc_means["baseline"], color="red", label="baseline")
plt.legend()
plt.show()
fig.savefig(
    f'../plots/{dataset_name}/Accuracies_Edges_One_Plot_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.png',
    bbox_inches="tight")  # save the figure to file
plt.close(fig)  

In [36]:
colors = ["blue", "green", "cyan", "purple", "orange", "brown", "magenta", ]
# alpha = 0.5
fig = plt.figure(figsize=(6.4, 6.4), layout="constrained")
ax = fig.add_subplot(111)
# ax.set_title("Test Accuracies der Strategien in Neuronenperspektive")
ax.set_xlabel("Epoch")
ax.set_ylabel("Accuracy (%)")
ax.set_xlim((0, max_epoch))
ax.set_ylim((0, 100))
curr_strat_index = 0
for i, strategy in enumerate(acc_means):  # iterates over keys which are the same for each result dict
    if "neurons" in strategy:
        ax.plot(np.arange(1, len(acc_means[strategy]) + 1), acc_means[strategy], label=strategy,
                color=colors[curr_strat_index], alpha=0.5)  #, alpha= alpha
        curr_strat_index += 1
        # if alpha == 0.5:
        #     alpha -= 0.25
        # else:
        #     alpha += 0.25
ax.plot(np.arange(1, len(acc_means["baseline"]) + 1), acc_means["baseline"], color="red", label="baseline")
plt.legend()
plt.show()
fig.savefig(
    f'../plots/{dataset_name}/Accuracies_Neurons_One_Plot_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.png',
    bbox_inches="tight")  # save the figure to file
plt.close(fig) 

In [22]:
num_layers = num_params_layers_expansions_means["random_edges"].shape[1]
fig = plt.figure(figsize=(4 * 6.4, 4 * 4.8), layout="constrained")  # 4 * 6.4, 3 * 4.8
fig.suptitle('Weight number increase among the different layers and expansions', fontsize=16)

for i, strategy in enumerate(acc_means, start=1):  # iterates over keys which are the same for each result dict
    if strategy != "baseline":
        ax_i = fig.add_subplot(4, 4, i)  # 3, 4, i
        ax_i.set_title(strategy)
        ax_i.set_xlabel("Layer")
        ax_i.set_ylabel("number of weights")
        #ax_i.set_xlim((0, max_epoch))
        ax_i.set_ylim((0, 27580))
        bottom = np.zeros(num_layers)
        width = 0.5
        for expansion in range(num_expansions + 1):
            ax_i.bar(np.arange(1, num_layers + 1), num_params_layers_expansions_means[strategy][expansion, :], width,
                     label=f"expansion {expansion}", bottom=bottom)
            bottom += num_params_layers_expansions_means[strategy][expansion, :]
            #print(bottom.sum())
        ax_i.legend(loc="upper right")
    plt.legend()
plt.show()
fig.savefig(
    f'../plots/{dataset_name}/Weight_number_increase_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.pdf',
    bbox_inches="tight")  # save the figure to file
plt.close(fig)  

In [44]:
num_layers = num_params_layers_expansions_means["random_edges"].shape[1]
fig = plt.figure(figsize=(3 * 6.4, 1 * 4.8), layout="constrained")  # 4 * 6.4, 3 * 4.8
# fig.suptitle(f'Anzahl hinzugefügter Parameter zu jeder Schicht über alle Expansionen ({dataset_name})', fontsize=16)

j=1
for i, strategy in enumerate(acc_means, start=1):  # iterates over keys which are the same for each result dict
    if "initial_edges" in strategy or "random_edges" in strategy or "warmstarted_edges" in strategy:
        ax_i = fig.add_subplot(1, 3, j)  # 3, 4, i
        j+=1
        ax_i.set_title(strategy)
        ax_i.set_xlabel("Layer")
        ax_i.set_ylabel("Anzahl der Parameter")
        #ax_i.set_xlim((0, max_epoch))
        ax_i.set_ylim((0, 27580))
        bottom = np.zeros(num_layers)
        width = 0.5
        for expansion in range(num_expansions + 1):
            ax_i.bar(np.arange(1, num_layers + 1), num_params_layers_expansions_means[strategy][expansion, :], width,
                     label=f"expansion {expansion}", bottom=bottom)
            bottom += num_params_layers_expansions_means[strategy][expansion, :]
            #print(bottom.sum())
        ax_i.legend(loc="upper right")
    plt.legend()
plt.show()
fig.savefig(
    f'../plots/{dataset_name}/Weight_number_increase_good_edgeStrats_seed_1_to_{num_seeds}_initial_network_{initial_network_id}.png',
    bbox_inches="tight")  # save the figure to file
plt.close(fig)  

In [24]:
num_layers = num_params_layers_expansions_means["random_edges"].shape[1]
fig = plt.figure(figsize=(1 * 6.4, 1 * 4.8), layout="constrained")  # 4 * 6.4, 3 * 4.8
# fig.suptitle(f'Anzahl hinzugefügter Parameter zu jeder Schicht über alle Expansionen ({dataset_name})', fontsize=16)

j=1
for i, strategy in enumerate(acc_means, start=1):  # iterates over keys which are the same for each result dict
    if "gradient_based_edges"  in strategy:
        ax_i = fig.add_subplot(1, 1, j)  # 3, 4, i
        j+=1
        ax_i.set_title(strategy)
        ax_i.set_xlabel("Layer")
        ax_i.set_ylabel("Anzahl der Parameter")
        #ax_i.set_xlim((0, max_epoch))
        ax_i.set_ylim((0, 27580))
        bottom = np.zeros(num_layers)
        width = 0.5
        for expansion in range(num_expansions + 1):
            ax_i.bar(np.arange(1, num_layers + 1), num_params_layers_expansions_means[strategy][expansion, :], width,
                     label=f"expansion {expansion}", bottom=bottom)
            bottom += num_params_layers_expansions_means[strategy][expansion, :]
            #print(bottom.sum())
        ax_i.legend(loc="upper right")
        plt.legend()
plt.show()
fig.savefig(
    f'../plots/{dataset_name}/Weight_number_increase_bad_edgeStrat_seed_1_to_{num_seeds}_initial_network_{initial_network_id}.png',
    bbox_inches="tight")  # save the figure to file
plt.close(fig)

In [40]:
num_layers = num_params_layers_expansions_means["random_edges"].shape[1]
fig = plt.figure(figsize=(2 * 6.4, 1 * 4.8), layout="constrained")  # 4 * 6.4, 3 * 4.8
# fig.suptitle(f'Anzahl hinzugefügter Parameter zu jeder Schicht über alle Expansionen ({dataset_name})', fontsize=16)

j=1
for i, strategy in enumerate(acc_means, start=1):  # iterates over keys which are the same for each result dict
    if "constant_neurons" in strategy or "regularized_neurons" in strategy:
        ax_i = fig.add_subplot(1, 2, j)  # 3, 4, i
        j+=1
        ax_i.set_title(strategy)
        ax_i.set_xlabel("Layer")
        ax_i.set_ylabel("Anzahl der Parameter")
        #ax_i.set_xlim((0, max_epoch))
        ax_i.set_ylim((0, 27580))
        bottom = np.zeros(num_layers)
        width = 0.5
        for expansion in range(num_expansions + 1):
            ax_i.bar(np.arange(1, num_layers + 1), num_params_layers_expansions_means[strategy][expansion, :], width,
                     label=f"expansion {expansion}", bottom=bottom)
            bottom += num_params_layers_expansions_means[strategy][expansion, :]
            #print(bottom.sum())
        ax_i.legend(loc="upper right")
        plt.legend()
plt.show()
fig.savefig(
    f'../plots/{dataset_name}/Weight_number_increase_good_and_bad_neuronStrats_seed_1_to_{num_seeds}_initial_network_{initial_network_id}.png',
    bbox_inches="tight")  # save the figure to file
plt.close(fig)

## Loading Testdata and evaluating the test data (ROC-curve, precision-recall-curve, f1-score)

In [41]:
from sklearn.preprocessing import LabelBinarizer

#load training data
#
# Load MINST dataset
#
input_path = '../MNIST/'
training_images_filepath = join(input_path, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
test_images_filepath = join(input_path, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
test_labels_filepath = join(input_path, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')
mnist_dataloader = mm.MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath,
                                      test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()

# convert to tensor
X_train_mnist = torch.Tensor(np.array(x_train)).to(device)
y_train_mnist = torch.Tensor(y_train).to(torch.long).to(device)
X_test_mnist = torch.Tensor(np.array(x_test)).to(device)
y_test_mnist = torch.Tensor(y_test).to(torch.long).to(device)

In [42]:
# Create datasets for training & validation, download if necessary
training_set_fashion_mnist = torchvision.datasets.FashionMNIST('../Fashion_MNIST', train=True,
                                                               transform=transforms.ToTensor(), download=True)
test_set_fashion_mnist = torchvision.datasets.FashionMNIST('../Fashion_MNIST', train=False,
                                                           transform=transforms.ToTensor(),
                                                           download=True)

X_train_fashion_mnist = training_set_fashion_mnist.data.to(device).to(torch.float32)
y_train_fashion_mnist = training_set_fashion_mnist.targets.to(device).to(torch.long)

X_test_fashion_mnist = test_set_fashion_mnist.data.to(device).to(torch.float32)
y_test_fashion_mnist = test_set_fashion_mnist.targets.to(device).to(torch.long)

In [43]:
label_binarizer_mnist = LabelBinarizer().fit(y_train_mnist.cpu().numpy())
y_onehot_test_mnist = label_binarizer_mnist.transform(y_test_mnist.cpu().numpy())

label_binarizer_fashion_mnist = LabelBinarizer().fit(y_train_fashion_mnist.cpu().numpy())
y_onehot_test_fashion_mnist = label_binarizer_fashion_mnist.transform(y_test_fashion_mnist.cpu().numpy())

# ROC-Curve

In [44]:
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import auc as sklearn_auc
from sklearn.metrics import roc_curve as sklearn_roc_curve

In [45]:
def compute_avg_curve(curves_x, curves_y, interval=(0, 1)):  #list of defining coordinate-lists respectively
    x_val_grid = np.linspace(interval[0], interval[1], 1001 * (interval[1] - interval[0]))
    mean_curve = np.zeros_like(x_val_grid)
    for i in range(len(curves_x)):
        mean_curve += np.interp(x_val_grid, curves_x[i], curves_y[i])
    mean_curve /= len(curves_x)
    return x_val_grid, mean_curve

In [46]:
def compute_macro_avg_roc_curve(y_onehot_test, y_scores):
    # store the fpr, tpr, and roc_auc for all OvR-classification problems
    fpr, tpr = list(), list()
    eps = 1e-9
    # is_sorted = lambda a: np.all(a[:-1] <= a[1:])
    n_classes = y_scores.shape[1]
    for i in range(n_classes):
        fpr_i, tpr_i, _ = sklearn_roc_curve(y_onehot_test[:, i], y_scores[:, i])
        fpr_i += (np.cumsum(np.ones(np.size(fpr_i))) - 1) * eps
        fpr.append(fpr_i)
        tpr.append(tpr_i)

    fpr_grid, mean_tpr = compute_avg_curve(fpr, tpr)
    return fpr_grid, mean_tpr

In [47]:
def get_score_vec(X_test, state_dict, device):
    model = mm.ProgressiveMLP(n_layers=6, n_neurons=30).to(device)

    model.load_state_dict(state_dict)  # state dict after 1st expansion

    y_test_pred = model(X_test)[-1]  #unnormalized logits

    # print(y_test_pred.size())

    softmax = torch.nn.Softmax(dim=1)

    y_scores_tensor = softmax(y_test_pred)  # actual class probabilities

    y_scores = y_scores_tensor.detach().cpu().numpy()
    return y_scores

In [48]:
strategies_roc_dict = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline":
        strategies_roc_dict[strategy] = []

for strategy in strategies_roc_dict:
    for expansion in range(num_expansions + 1):
        tpr_list, fpr_list = [], []
        for seed in range(num_seeds):
            if dataset_name == fashion_mnist_name:
                y_scores = get_score_vec(X_test_fashion_mnist,
                                         strategies_result_dict[strategy][seed]["weights"][expansion], device)
                roc_curve = compute_macro_avg_roc_curve(y_onehot_test_fashion_mnist, y_scores)  # fashion_mnist
            else:
                y_scores = get_score_vec(X_test_mnist, strategies_result_dict[strategy][seed]["weights"][expansion],
                                         device)
                roc_curve = compute_macro_avg_roc_curve(y_onehot_test_mnist, y_scores)  # mnist
            fpr_list.append(roc_curve[0])
            tpr_list.append(roc_curve[1])
        fpr, tpr = compute_avg_curve(fpr_list, tpr_list)
        strategies_roc_dict[strategy].append((fpr, tpr))

In [49]:
display_padding = 1e-2
for strategy in strategies_roc_dict:
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(1, 1, 1)
    for expansion in range(num_expansions + 1):
        ax.plot(
            strategies_roc_dict[strategy][expansion][0],
            strategies_roc_dict[strategy][expansion][1],
            label=f"Expansion {expansion} (AUC = {sklearn_auc(strategies_roc_dict[strategy][expansion][0], strategies_roc_dict[strategy][expansion][1]):.3f})",
            linestyle="solid",
            linewidth=1.5,
            alpha=0.5
        )
    ax.plot(
        [0, 1],
        [0, 1],
        label="Chance level (AUC = 0.5)",
        color="black",
        linestyle="dashed",
        linewidth=1.5
    )
    ax.set(
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"Macro-averaged ROC-Curves\nfor {strategy}",
        xlim=(0 - display_padding, 1 + display_padding),
        ylim=(0 - display_padding, 1 + display_padding)
    )
    plt.legend()
    fig.savefig(
        f'../plots/{dataset_name}/{strategy}_ROC_curve_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.pdf',
        bbox_inches="tight")  # save the figure to file
    plt.close(fig)  # close the figure window


In [50]:
colors = ["blue", "blue", "green", "green", "cyan", "cyan", "purple", "purple", "orange", "orange", "brown", "brown",
          "magenta", "magenta"]
display_padding = 1e-2
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1)
for i, strategy in enumerate(strategies_roc_dict):
    if "edges" in strategy:
        ax.plot(
            strategies_roc_dict[strategy][1][0],
            strategies_roc_dict[strategy][1][1],
            label=f"Strategy {strategy} (AUC = {sklearn_auc(strategies_roc_dict[strategy][1][0], strategies_roc_dict[strategy][1][1]):.3f})",
            linestyle="solid",
            color=colors[i],
            linewidth=1.5,
            alpha=0.5
        )
ax.plot(
    [0, 1],
    [0, 1],
    label="Chance level (AUC = 0.5)",
    color="black",
    linestyle="dashed",
    linewidth=1.5
)
ax.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    # title=f"Macro-Average der ROC-Kurven der Kantenstrategien\nnach der ersten Expansion",
    xlim=(0 - display_padding, 1 + display_padding),
    ylim=(0 - display_padding, 1 + display_padding)
)
plt.legend()
fig.savefig(
    f'../plots/{dataset_name}/edge_strategies_after_exp1_ROC_curve_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.png',
    bbox_inches="tight")  # save the figure to file
plt.close(fig)  # close the figure window


In [51]:
display_padding = 1e-2
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1)
for i, strategy in enumerate(strategies_roc_dict):
    if "neurons" in strategy:
        ax.plot(
            strategies_roc_dict[strategy][1][0],
            strategies_roc_dict[strategy][1][1],
            label=f"Strategy {strategy} (AUC = {sklearn_auc(strategies_roc_dict[strategy][1][0], strategies_roc_dict[strategy][1][1]):.3f})",
            linestyle="solid",
            color=colors[i],
            linewidth=1.5,
            alpha=0.5
        )
ax.plot(
    [0, 1],
    [0, 1],
    label="Chance level (AUC = 0.5)",
    color="black",
    linestyle="dashed",
    linewidth=1.5
)
ax.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    # title=f"Macro-Average der ROC-Kurven der Neuronenstrategien\nnach der ersten Expansion",
    xlim=(0 - display_padding, 1 + display_padding),
    ylim=(0 - display_padding, 1 + display_padding)
)
plt.legend()
fig.savefig(
    f'../plots/{dataset_name}/neuron_strategies_after_exp1_ROC_curve_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.png',
    bbox_inches="tight")  # save the figure to file
plt.close(fig)  # close the figure window

# F1-score (harmonic mean of precision and recall)

In [20]:
from sklearn.metrics import f1_score as sklearn_f1_score

strategies_f1_score_dict = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline":
        strategies_f1_score_dict[strategy] = []

for strategy in strategies_f1_score_dict:
    for expansion in range(num_expansions + 1):
        f1_list = []
        for seed in range(num_seeds):
            if dataset_name == fashion_mnist_name:
                y_true = y_test_fashion_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_fashion_mnist,
                                         strategies_result_dict[strategy][seed]["weights"][expansion], device)
            else:
                y_true = y_test_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_mnist, strategies_result_dict[strategy][seed]["weights"][expansion],
                                         device)
            y_pred = np.argmax(y_scores, axis=1)
            f1_list.append(sklearn_f1_score(y_true, y_pred, average="macro"))
        strategies_f1_score_dict[strategy].append((np.mean(f1_list), np.std(f1_list)))

# prepare to read in as pandas dataframe
table_numpy = np.zeros((len(strategies_f1_score_dict), num_expansions + 1)).astype(np.str_)
for i, strategy in enumerate(strategies_f1_score_dict):
    for expansion in range(num_expansions + 1):
        table_numpy[
            i, expansion] = f"{strategies_f1_score_dict[strategy][expansion][0]:.2f} ± {strategies_f1_score_dict[strategy][expansion][1]:.2f}"

df = pd.DataFrame(table_numpy, index=strategies_f1_score_dict.keys(),
                  columns=np.arange(num_expansions + 1)).rename_axis(index="Strategie", columns="Erweiterung")

with open(
        f"../tables/{dataset_name}/F1_Score_latex_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.txt",
        "w") as text_file:
    print(df.to_latex(), file=text_file)

In [21]:
strategies_f1_score_dict = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline" and "neurons" not in strategy:
        strategies_f1_score_dict[strategy] = []

for strategy in strategies_f1_score_dict:
    for expansion in range(num_expansions + 1):
        f1_list = []
        for seed in range(num_seeds):
            if dataset_name == fashion_mnist_name:
                y_true = y_test_fashion_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_fashion_mnist,
                                         strategies_result_dict[strategy][seed]["weights"][expansion], device)
            else:
                y_true = y_test_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_mnist, strategies_result_dict[strategy][seed]["weights"][expansion],
                                         device)
            y_pred = np.argmax(y_scores, axis=1)
            f1_list.append(sklearn_f1_score(y_true, y_pred, average="macro"))
        strategies_f1_score_dict[strategy].append((np.mean(f1_list), np.std(f1_list)))

# prepare to read in as pandas dataframe
table_numpy = np.zeros((len(strategies_f1_score_dict), num_expansions + 1)).astype(np.str_)
for i, strategy in enumerate(strategies_f1_score_dict):
    for expansion in range(num_expansions + 1):
        table_numpy[
            i, expansion] = f"{strategies_f1_score_dict[strategy][expansion][0]:.2f} ± {strategies_f1_score_dict[strategy][expansion][1]:.2f}"

df = pd.DataFrame(table_numpy, index=strategies_f1_score_dict.keys(),
                  columns=np.arange(num_expansions + 1)).rename_axis(index="Strategie", columns="Erweiterung")

with open(
        f"../tables/{dataset_name}/F1_Score_edges_seed_1_to_{num_seeds}_initial_network_{initial_network_id}.txt",
        "w") as text_file:
    print(df.to_latex(), file=text_file)

In [22]:
strategies_f1_score_dict = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline" and "edges" not in strategy:
        strategies_f1_score_dict[strategy] = []

for strategy in strategies_f1_score_dict:
    for expansion in range(num_expansions + 1):
        f1_list = []
        for seed in range(num_seeds):
            if dataset_name == fashion_mnist_name:
                y_true = y_test_fashion_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_fashion_mnist,
                                         strategies_result_dict[strategy][seed]["weights"][expansion], device)
            else:
                y_true = y_test_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_mnist, strategies_result_dict[strategy][seed]["weights"][expansion],
                                         device)
            y_pred = np.argmax(y_scores, axis=1)
            f1_list.append(sklearn_f1_score(y_true, y_pred, average="macro"))
        strategies_f1_score_dict[strategy].append((np.mean(f1_list), np.std(f1_list)))

# prepare to read in as pandas dataframe
table_numpy = np.zeros((len(strategies_f1_score_dict), num_expansions + 1)).astype(np.str_)
for i, strategy in enumerate(strategies_f1_score_dict):
    for expansion in range(num_expansions + 1):
        table_numpy[
            i, expansion] = f"{strategies_f1_score_dict[strategy][expansion][0]:.2f} ± {strategies_f1_score_dict[strategy][expansion][1]:.2f}"

df = pd.DataFrame(table_numpy, index=strategies_f1_score_dict.keys(),
                  columns=np.arange(num_expansions + 1)).rename_axis(index="Strategie", columns="Erweiterung")

with open(
        f"../tables/{dataset_name}/F1_Score_neurons_seed_1_to_{num_seeds}_initial_network_{initial_network_id}.txt",
        "w") as text_file:
    print(df.to_latex(), file=text_file)

# Accuracy

In [23]:
def accuracy(y_true, y_pred):
    n_correct = np.count_nonzero(y_true == y_pred)
    return n_correct / y_true.size

In [24]:
strategies_test_accuracy_at_expansion = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline":
        strategies_test_accuracy_at_expansion[strategy] = []

for strategy in strategies_test_accuracy_at_expansion:
    for expansion in range(num_expansions + 1):
        acc_list = []
        for seed in range(num_seeds):
            if dataset_name == fashion_mnist_name:
                y_true = y_test_fashion_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_fashion_mnist,
                                         strategies_result_dict[strategy][seed]["weights"][expansion], device)
            else:
                y_true = y_test_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_mnist, strategies_result_dict[strategy][seed]["weights"][expansion],
                                         device)
            y_pred = np.argmax(y_scores, axis=1)
            acc_list.append(accuracy(y_true, y_pred))
        strategies_test_accuracy_at_expansion[strategy].append((np.mean(acc_list), np.std(acc_list)))

# prepare to read in as pandas dataframe
table_numpy = np.zeros((len(strategies_test_accuracy_at_expansion), num_expansions + 1)).astype(np.str_)
for i, strategy in enumerate(strategies_test_accuracy_at_expansion):
    for expansion in range(num_expansions + 1):
        table_numpy[
            i, expansion] = f"{strategies_test_accuracy_at_expansion[strategy][expansion][0]:.3f} ± {strategies_test_accuracy_at_expansion[strategy][expansion][1]:.3f}"

df = pd.DataFrame(table_numpy, index=strategies_test_accuracy_at_expansion.keys(),
                  columns=np.arange(num_expansions + 1)).rename_axis(index="Strategie", columns="Erweiterung")

with open(
        f"../tables/{dataset_name}/Accuracy_latex_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.txt",
        "w") as text_file:
    print(df.to_latex(), file=text_file)

In [25]:
strategies_test_accuracy_at_expansion = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline" and "neurons" not in strategy:
        strategies_test_accuracy_at_expansion[strategy] = []

for strategy in strategies_test_accuracy_at_expansion:
    for expansion in range(num_expansions + 1):
        acc_list = []
        for seed in range(num_seeds):
            if dataset_name == fashion_mnist_name:
                y_true = y_test_fashion_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_fashion_mnist,
                                         strategies_result_dict[strategy][seed]["weights"][expansion], device)
            else:
                y_true = y_test_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_mnist, strategies_result_dict[strategy][seed]["weights"][expansion],
                                         device)
            y_pred = np.argmax(y_scores, axis=1)
            acc_list.append(accuracy(y_true, y_pred))
        strategies_test_accuracy_at_expansion[strategy].append((np.mean(acc_list), np.std(acc_list)))

# prepare to read in as pandas dataframe
table_numpy = np.zeros((len(strategies_test_accuracy_at_expansion), num_expansions + 1)).astype(np.str_)
for i, strategy in enumerate(strategies_test_accuracy_at_expansion):
    for expansion in range(num_expansions + 1):
        table_numpy[
            i, expansion] = f"{strategies_test_accuracy_at_expansion[strategy][expansion][0]:.3f} ± {strategies_test_accuracy_at_expansion[strategy][expansion][1]:.3f}"

df = pd.DataFrame(table_numpy, index=strategies_test_accuracy_at_expansion.keys(),
                  columns=np.arange(num_expansions + 1)).rename_axis(index="Strategie", columns="Erweiterung")

with open(
        f"../tables/{dataset_name}/Accuracy_edges_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.txt",
        "w") as text_file:
    print(df.to_latex(), file=text_file)

In [26]:
strategies_test_accuracy_at_expansion = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline" and "edges" not in strategy:
        strategies_test_accuracy_at_expansion[strategy] = []

for strategy in strategies_test_accuracy_at_expansion:
    for expansion in range(num_expansions + 1):
        acc_list = []
        for seed in range(num_seeds):
            if dataset_name == fashion_mnist_name:
                y_true = y_test_fashion_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_fashion_mnist,
                                         strategies_result_dict[strategy][seed]["weights"][expansion], device)
            else:
                y_true = y_test_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_mnist, strategies_result_dict[strategy][seed]["weights"][expansion],
                                         device)
            y_pred = np.argmax(y_scores, axis=1)
            acc_list.append(accuracy(y_true, y_pred))
        strategies_test_accuracy_at_expansion[strategy].append((np.mean(acc_list), np.std(acc_list)))

# prepare to read in as pandas dataframe
table_numpy = np.zeros((len(strategies_test_accuracy_at_expansion), num_expansions + 1)).astype(np.str_)
for i, strategy in enumerate(strategies_test_accuracy_at_expansion):
    for expansion in range(num_expansions + 1):
        table_numpy[
            i, expansion] = f"{strategies_test_accuracy_at_expansion[strategy][expansion][0]:.3f} ± {strategies_test_accuracy_at_expansion[strategy][expansion][1]:.3f}"

df = pd.DataFrame(table_numpy, index=strategies_test_accuracy_at_expansion.keys(),
                  columns=np.arange(num_expansions + 1)).rename_axis(index="Strategie", columns="Erweiterung")

with open(
        f"../tables/{dataset_name}/Accuracy_neurons_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.txt",
        "w") as text_file:
    print(df.to_latex(), file=text_file)

# AUC (area under curve)

In [30]:
strategies_auc_dict = dict()
for strategy in strategies_roc_dict:
    strategies_auc_dict[strategy] = []

for strategy in strategies_auc_dict:
    for expansion in range(num_expansions + 1):
        strategies_auc_dict[strategy].append(sklearn_auc(*strategies_roc_dict[strategy][expansion]))

# prepare to read in as pandas dataframe
table_numpy = np.zeros((len(strategies_auc_dict), num_expansions + 1)).astype(np.str_)
for i, strategy in enumerate(strategies_test_accuracy_at_expansion):
    for expansion in range(num_expansions + 1):
        table_numpy[
            i, expansion] = f"{strategies_auc_dict[strategy][expansion]:.3f}"  #  ± {strategies_test_accuracy_at_expansion[strategy][expansion][1]:.3f}

df = pd.DataFrame(table_numpy, index=strategies_auc_dict.keys(), columns=np.arange(num_expansions + 1)).rename_axis(
    index="Strategie", columns="Erweiterung")

with open(
        f"../tables/{dataset_name}/AUC_latex_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.txt",
        "w") as text_file:
    print(df.to_latex(), file=text_file)


# Number of layer parameters across all expansions

In [27]:
num_layers = num_params_layers_expansions_means["random_edges"].shape[1]
strategies_num_params_among_layers_dict = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline":
        strategies_num_params_among_layers_dict[strategy] = []

for strategy in strategies_num_params_among_layers_dict:
    strategies_num_params_among_layers_dict[strategy] = np.ravel(
        num_params_layers_expansions_means[strategy][:2, :])  # hierarchy: expansions -> layers

mux_columns = pd.MultiIndex.from_product([np.arange(2), np.arange(1, num_layers + 1)],
                                         names=["Expansion", "Layer"])

# prepare to read in as pandas dataframe
table_numpy = np.zeros((len(strategies_num_params_among_layers_dict), (2) * num_layers)).astype(
    np.str_)
for i, strategy in enumerate(strategies_num_params_among_layers_dict):
    for j in range(table_numpy.shape[1]):
        table_numpy[
            i, j] = f"{int(strategies_num_params_among_layers_dict[strategy][j]):d}"  #  ± {strategies_test_accuracy_at_expansion[strategy][expansion][1]:.3f}

df = pd.DataFrame(table_numpy, index=strategies_num_params_among_layers_dict.keys(), columns=mux_columns).rename_axis(
    index="Strategie")

with open(
        f"../tables/{dataset_name}/Params_added_latex_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.txt",
        "w") as text_file:
    print(df.to_latex(), file=text_file)

In [28]:
num_layers = num_params_layers_expansions_means["random_edges"].shape[1]
strategies_num_params_among_layers_dict = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline" and "neurons" not in strategy:
        strategies_num_params_among_layers_dict[strategy] = []

for strategy in strategies_num_params_among_layers_dict:
    strategies_num_params_among_layers_dict[strategy] = np.ravel(
        num_params_layers_expansions_means[strategy][:2, :])  # hierarchy: expansions -> layers

mux_columns = pd.MultiIndex.from_product([np.arange(2), np.arange(1, num_layers + 1)],
                                         names=["Expansion", "Layer"])

# prepare to read in as pandas dataframe
table_numpy = np.zeros((len(strategies_num_params_among_layers_dict), (2) * num_layers)).astype(
    np.str_)
for i, strategy in enumerate(strategies_num_params_among_layers_dict):
    for j in range(table_numpy.shape[1]):
        table_numpy[
            i, j] = f"{int(strategies_num_params_among_layers_dict[strategy][j]):d}"  #  ± {strategies_test_accuracy_at_expansion[strategy][expansion][1]:.3f}

df = pd.DataFrame(table_numpy, index=strategies_num_params_among_layers_dict.keys(), columns=mux_columns).rename_axis(
    index="Strategie")

with open(
        f"../tables/{dataset_name}/Params_added_edges_seed_1_to_{num_seeds}_initial_network_{initial_network_id}.txt",
        "w") as text_file:
    print(df.to_latex(), file=text_file)

In [29]:
num_layers = num_params_layers_expansions_means["random_edges"].shape[1]
strategies_num_params_among_layers_dict = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline" and "edges" not in strategy:
        strategies_num_params_among_layers_dict[strategy] = []

for strategy in strategies_num_params_among_layers_dict:
    strategies_num_params_among_layers_dict[strategy] = np.ravel(
        num_params_layers_expansions_means[strategy][:2, :])  # hierarchy: expansions -> layers

mux_columns = pd.MultiIndex.from_product([np.arange(2), np.arange(1, num_layers + 1)],
                                         names=["Expansion", "Layer"])

# prepare to read in as pandas dataframe
table_numpy = np.zeros((len(strategies_num_params_among_layers_dict), (2) * num_layers)).astype(
    np.str_)
for i, strategy in enumerate(strategies_num_params_among_layers_dict):
    for j in range(table_numpy.shape[1]):
        table_numpy[
            i, j] = f"{int(strategies_num_params_among_layers_dict[strategy][j]):d}"  #  ± {strategies_test_accuracy_at_expansion[strategy][expansion][1]:.3f}

df = pd.DataFrame(table_numpy, index=strategies_num_params_among_layers_dict.keys(), columns=mux_columns).rename_axis(
    index="Strategie")

with open(
        f"../tables/{dataset_name}/Params_added_neurons_seed_1_to_{num_seeds}_initial_network_{initial_network_id}.txt",
        "w") as text_file:
    print(df.to_latex(), file=text_file)

## Connectivity Score (percentage of non bias weights that are connected to input and output layer) (old)

In [30]:
strategies_connectivity_dict = torch.load(
    "../results/" + f"Connectivity_scores_{dataset_name}_initial_network_{initial_network_id}_num_seeds_{num_seeds}_expansions_{num_expansions}_eps_{eps}_lr_{lr}_grad_clip_{grad_clip}.pt") #TODO

In [31]:
strategies_connectivity_dict_chosen = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline":
        strategies_connectivity_dict_chosen[strategy] = strategies_connectivity_dict[strategy]
# prepare to read in as pandas dataframe
table_numpy = np.zeros((len(strategies_connectivity_dict_chosen), num_expansions + 1)).astype(np.str_)
for i, strategy in enumerate(strategies_connectivity_dict_chosen):
    for expansion in range(num_expansions + 1):
        table_numpy[i, expansion] = f"{strategies_connectivity_dict[strategy][expansion]:.2f}"

df = pd.DataFrame(table_numpy, index=strategies_connectivity_dict_chosen.keys(),
                  columns=np.arange(num_expansions + 1)).rename_axis(index="Strategie", columns="Erweiterung")

with open(
        f"../tables/{dataset_name}/Connectivity_score_latex_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.txt",
        "w") as text_file:
    print(df.to_latex(), file=text_file)

In [32]:
strategies_connectivity_dict_chosen = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline" and "neurons" not in strategy:
        strategies_connectivity_dict_chosen[strategy] = strategies_connectivity_dict[strategy]
# prepare to read in as pandas dataframe
table_numpy = np.zeros((len(strategies_connectivity_dict_chosen), num_expansions + 1)).astype(np.str_)
for i, strategy in enumerate(strategies_connectivity_dict_chosen):
    for expansion in range(num_expansions + 1):
        table_numpy[i, expansion] = f"{strategies_connectivity_dict[strategy][expansion]:.2f}"

df = pd.DataFrame(table_numpy, index=strategies_connectivity_dict_chosen.keys(),
                  columns=np.arange(num_expansions + 1)).rename_axis(index="Strategie", columns="Erweiterung")

with open(
        f"../tables/{dataset_name}/Connectivity_score_edges_seed_1_to_{num_seeds}_initial_network_{initial_network_id}.txt",
        "w") as text_file:
    print(df.to_latex(), file=text_file)

In [33]:
strategies_connectivity_dict_chosen = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline" and "edges" not in strategy:
        strategies_connectivity_dict_chosen[strategy] = strategies_connectivity_dict[strategy]
# prepare to read in as pandas dataframe
table_numpy = np.zeros((len(strategies_connectivity_dict_chosen), num_expansions + 1)).astype(np.str_)
for i, strategy in enumerate(strategies_connectivity_dict_chosen):
    for expansion in range(num_expansions + 1):
        table_numpy[i, expansion] = f"{strategies_connectivity_dict[strategy][expansion]:.2f}"

df = pd.DataFrame(table_numpy, index=strategies_connectivity_dict_chosen.keys(),
                  columns=np.arange(num_expansions + 1)).rename_axis(index="Strategie", columns="Erweiterung")

with open(
        f"../tables/{dataset_name}/Connectivity_score_neurons_seed_1_to_{num_seeds}_initial_network_{initial_network_id}.txt",
        "w") as text_file:
    print(df.to_latex(), file=text_file)

# Confusion Matrix computation (after 1st expansion)

In [34]:
from sklearn.metrics import confusion_matrix as sklearn_confusion_matrix

strategies_confusion_matrix_at_expansion = dict()
for strategy in strategies_result_dict.keys():
    if strategy != "baseline":
        strategies_confusion_matrix_at_expansion[strategy] = []

for strategy in strategies_confusion_matrix_at_expansion:
    for expansion in range(num_expansions + 1):
        confusion_matrix_list = []
        for seed in range(num_seeds):
            if dataset_name == fashion_mnist_name:
                y_true = y_test_fashion_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_fashion_mnist,
                                         strategies_result_dict[strategy][seed]["weights"][expansion], device)
            else:
                y_true = y_test_mnist.cpu().numpy()
                y_scores = get_score_vec(X_test_mnist, strategies_result_dict[strategy][seed]["weights"][expansion],
                                         device)
            y_pred = np.argmax(y_scores, axis=1)
            confusion_matrix_list.append(sklearn_confusion_matrix(y_true, y_pred, normalize="all"))
        strategies_confusion_matrix_at_expansion[strategy].append(np.mean(np.array(confusion_matrix_list), axis=0))  # np.round( , decimals=2)

In [35]:
# save as confusion matrix as heatmap
# Class labels fashion mnist
class_names_fashion_mnist = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')
for strategy in strategies_confusion_matrix_at_expansion:
    if dataset_name == fashion_mnist_name:
        df_cm = pd.DataFrame(strategies_confusion_matrix_at_expansion[strategy][1], index = class_names_fashion_mnist,
                      columns = class_names_fashion_mnist).rename_axis(index="Actual", columns="Predicted")
    else:
        df_cm = pd.DataFrame(strategies_confusion_matrix_at_expansion[strategy][1], index = [i for i in range(10)],
                      columns = [i for i in range(10)]).rename_axis(index="Actual", columns="Predicted")
    fig = plt.figure(figsize=(10, 7), layout="constrained")
    ax = fig.add_subplot(1, 1, 1)
    sn.heatmap(df_cm, annot=True, ax=ax)
    fig.savefig(
        f'../plots/{dataset_name}/{strategy}_confusion_matrix_at_1stExpansion_seed_1_to_{num_seeds}_expansions_{num_expansions}_initial_network_{initial_network_id}.pdf',
        bbox_inches="tight")  # save the figure to file
    plt.close(fig)  # close the figure window